# Best and popular movies

In [67]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('MovieLens').getOrCreate()

## Reading in the data

In [68]:
ratings = spark.read.option("header", "true").csv("data/ratings.csv")
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    110|   1.0|1425941529|
|     1|    147|   4.5|1425942435|
|     1|    858|   5.0|1425941523|
|     1|   1221|   5.0|1425941546|
|     1|   1246|   5.0|1425941556|
+------+-------+------+----------+


In [69]:
movies = spark.read.option("header", "true").csv("data/movies_metadata.csv").select(col("id"), col("title"))
movies.show(5)

+-----+--------------------+
|   id|               title|
+-----+--------------------+
|  862|           Toy Story|
| 8844|             Jumanji|
|15602|    Grumpier Old Men|
|31357|[{'iso_639_1': 'e...|
|11862|Father of the Bri...|
+-----+--------------------+


## Most popular movies

In [70]:
most_popular = ratings\
.groupBy("movieId")\
.agg(count("userId"))\
.withColumnRenamed("count(userId)", "num_ratings")\
.sort(desc("num_ratings"))

In [71]:
most_popular_movies = most_popular.join(movies, most_popular.movieId == movies.id)
most_popular_movies.show(10)

+-------+-----------+-----+--------------------+
|movieId|num_ratings|   id|               title|
+-------+-----------+-----+--------------------+
|    296|      87901|  296|Terminator 3: Ris...|
|   2294|      12526| 2294|Jay and Silent Bo...|
|   3210|       9516| 3210|   The Projected Man|
|   1090|      18222| 1090|The Thirteenth Floor|
|   2162|       2365| 2162|        Chill Factor|
|    829|       1961|  829|           Chinatown|
|    467|        804|  467|            The Hole|
|    691|       1262|  691|The Spy Who Loved Me|
|   2088|       3841| 2088|   Romeo Is Bleeding|
|  27317|       1577|27317|Patton Oswalt: My...|
+-------+-----------+-----+--------------------+


## Top rated movies

In [72]:
top_rated = ratings\
.groupBy("movieId")\
.agg(avg(col("rating")))\
.withColumnRenamed("avg(rating)", "avg_rating")\
.sort(desc("avg_rating"))

In [73]:
top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.id)
top_rated_movies.show(10)

+-------+------------------+-----+--------------------+
|movieId|        avg_rating|   id|               title|
+-------+------------------+-----+--------------------+
|    296| 4.169975313136369|  296|Terminator 3: Ris...|
|   2294|3.2492016605460643| 2294|Jay and Silent Bo...|
|   3210|3.6424443043295502| 3210|   The Projected Man|
|   1090|3.9067061793436504| 1090|The Thirteenth Floor|
|   2162| 2.483720930232558| 2162|        Chill Factor|
|    829| 2.684344722080571|  829|           Chinatown|
|    467|3.4284825870646767|  467|            The Hole|
|    691| 3.053090332805071|  691|The Spy Who Loved Me|
|   2088|2.5669096589429836| 2088|   Romeo Is Bleeding|
|  27317|  3.60145846544071|27317|Patton Oswalt: My...|
+-------+------------------+-----+--------------------+


In [74]:
top_rated = ratings\
.groupBy("movieId")\
.agg(count("userId"), avg(col("rating")))\
.withColumnRenamed("count(userId)", "num_ratings")\
.withColumnRenamed("avg(rating)", "avg_rating")

In [75]:
top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.id).sort(desc("avg_rating"), desc("num_ratings"))
top_rated_movies.show(10)

+-------+-----------+----------+------+--------------------+
|movieId|num_ratings|avg_rating|    id|               title|
+-------+-----------+----------+------+--------------------+
|  95977|          1|       5.0| 95977|The Man Behind Th...|
| 132912|          1|       5.0|132912|    The Price of Sex|
| 137853|          1|       5.0|137853| Journey to Planet X|
| 130544|          1|       5.0|130544|Palermo or Wolfsburg|
| 146946|          1|       5.0|146946|         Yellow Rock|
| 173153|          1|       5.0|173153|        Phil Spector|
| 160329|          1|       5.0|160329|         Lightheaded|
| 164278|          1|       5.0|164278|              Harvey|
| 169726|          1|       5.0|169726|The Gypsy and the...|
| 166231|          1|       5.0|166231|             Refugee|
+-------+-----------+----------+------+--------------------+


In [76]:
# Calculate average, minimum, and maximum of num_ratings
top_rated_movies.select([mean('num_ratings'), min('num_ratings'), max('num_ratings')]).show(1)

+------------------+----------------+----------------+
|  avg(num_ratings)|min(num_ratings)|max(num_ratings)|
+------------------+----------------+----------------+
|1504.8917020148463|               1|           91082|
+------------------+----------------+----------------+


In [77]:
top_rated_movies.where("num_ratings > 500").show(20, truncate=False)

+-------+-----------+------------------+-----+--------------------------------------------+
|movieId|num_ratings|avg_rating        |id   |title                                       |
+-------+-----------+------------------+-----+--------------------------------------------+
|318    |91082      |4.429014514393623 |318  |The Million Dollar Hotel                    |
|858    |57070      |4.339810758717364 |858  |Sleepless in Seattle                        |
|527    |67662      |4.266530696698294 |527  |Once Were Warriors                          |
|2019   |13994      |4.255073602972702 |2019 |Hard Target                                 |
|2959   |60024      |4.2307160469145675|2959 |First came love... then came Reverend Frank.|
|912    |30043      |4.2143927037912325|912  |The Thomas Crown Affair                     |
|750    |28280      |4.213030410183875 |750  |Murder She Said                             |
|5618   |20855      |4.202589307120594 |5618 |Cousin, Cousine                   